# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from random import sample

import re

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [3]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/sales_train_validation.csv')
calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/calendar.csv')

### Note: This Step is for the extraction of California State and Foods category 

In [4]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOUSEHOLD')]

In [5]:
sales_train_val

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
565,HOUSEHOLD_1_001_CA_1_validation,HOUSEHOLD_1_001,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,3,0,0,0,...,1,0,1,1,1,0,0,0,0,1
566,HOUSEHOLD_1_002_CA_1_validation,HOUSEHOLD_1_002,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,1,1,2,4,...,0,0,0,0,0,1,0,0,0,1
567,HOUSEHOLD_1_003_CA_1_validation,HOUSEHOLD_1_003,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,0,0,1,2,1,0,0,0,1,0
568,HOUSEHOLD_1_004_CA_1_validation,HOUSEHOLD_1_004,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,4,0,8,11,...,0,5,0,0,0,0,0,0,1,0
569,HOUSEHOLD_1_005_CA_1_validation,HOUSEHOLD_1_005,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,3,2,2,...,1,0,3,1,2,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10754,HOUSEHOLD_2_512_CA_4_validation,HOUSEHOLD_2_512,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,0,0,0,0,2,1,0,0,0,0
10755,HOUSEHOLD_2_513_CA_4_validation,HOUSEHOLD_2_513,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
10756,HOUSEHOLD_2_514_CA_4_validation,HOUSEHOLD_2_514,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,2,0,1,...,0,0,1,0,0,0,0,0,0,0
10757,HOUSEHOLD_2_515_CA_4_validation,HOUSEHOLD_2_515,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0


In [6]:
INPUT_DIR_2 = r'/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/List of Product ID according to 4 demand patterns/California/Household'

list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

In [7]:
len(sales_intermittent)

1493

In [8]:
len(sales_lumpy)

178

In [9]:
len(sales_erratic)

126

In [10]:
len(sales_smooth)

2391

### Check by looking at column "id" to verify if we choose the right product (FOODS or HOBBIES or HOUSEHOLD)

In [11]:
sales_smooth

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
567,HOUSEHOLD_1_003_CA_1_validation,HOUSEHOLD_1_003,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,0,0,1,2,1,0,0,0,1,0
568,HOUSEHOLD_1_004_CA_1_validation,HOUSEHOLD_1_004,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,4,0,8,11,...,0,5,0,0,0,0,0,0,1,0
569,HOUSEHOLD_1_005_CA_1_validation,HOUSEHOLD_1_005,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,3,2,2,...,1,0,3,1,2,0,0,1,0,1
570,HOUSEHOLD_1_006_CA_1_validation,HOUSEHOLD_1_006,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,3,4,2,6,3,2,2,1,3,4
571,HOUSEHOLD_1_007_CA_1_validation,HOUSEHOLD_1_007,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,0,3,1,1,1,3,0,5,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10742,HOUSEHOLD_2_500_CA_4_validation,HOUSEHOLD_2_500,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,0,1,0,1,0,0,1,0,0,1
10747,HOUSEHOLD_2_505_CA_4_validation,HOUSEHOLD_2_505,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
10752,HOUSEHOLD_2_510_CA_4_validation,HOUSEHOLD_2_510,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,0,0,0,2,1,1,0,1,0,0
10753,HOUSEHOLD_2_511_CA_4_validation,HOUSEHOLD_2_511,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,2,1,...,1,1,1,0,0,3,1,0,0,1


# User-defined functions to calculate Metrics and Croston algorithm

In [12]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

In [13]:
def Croston(ts,extra_periods=1,alpha=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), periodicity(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
    q = 1 #periods since last demand observation
    
    # Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1 + first_occurence
    f[0] = a[0]/p[0]
    # Create all the t+1 forecasts
    for t in range(0,cols):        
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = alpha*q + (1-alpha)*p[t]
            f[t+1] = a[t+1]/p[t+1]
            q = 1           
        else:
            a[t+1] = a[t]
            p[t+1] = p[t]
            f[t+1] = f[t]
            q += 1
       
    # Future Forecast 
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

# Format sales data and then merge with calendar data

In [14]:
sales_pattern = sales_smooth.copy()

In [15]:
sales_pattern_py = sales_pattern.copy()

list_pattern_py = sales_pattern_py.id.unique().tolist()
list_pattern_py = [(re.sub(r'_validation', r"", x)) for x in list_pattern_py]

sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
df_pattern.drop('Date', axis=1, inplace=True)
df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
df_pattern.index.names = ['Date']
df_pattern = df_pattern.astype('float64')

df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
df_pattern.reset_index(inplace=True)

calendar['Date'] = pd.to_datetime(calendar['date'])

dataframe = pd.merge(df_pattern,
                     calendar[['wm_yr_wk', 'Date']],
                     on = 'Date')

not_enough_seven_days = dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})[dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})['Date'] < 7].index.tolist()
dataframe = dataframe[~dataframe['wm_yr_wk'].isin(not_enough_seven_days)]

dataframe.set_index('wm_yr_wk', inplace=True)
dataframe.drop('Date', axis=1, inplace=True)

dataframe_weekly = dataframe.groupby('wm_yr_wk').sum().reset_index()

# Set value for parameters

In [16]:
list_params = [round(item, 2) for item in list(np.arange(0.05, 0.95, 0.05))]

In [17]:
list_params

[0.05,
 0.1,
 0.15,
 0.2,
 0.25,
 0.3,
 0.35,
 0.4,
 0.45,
 0.5,
 0.55,
 0.6,
 0.65,
 0.7,
 0.75,
 0.8,
 0.85,
 0.9]

In [18]:
start_train_week = 11318
end_train_week = 11610
n_pred_weeks = 2
num_train_needed = 50
validation_training_ratio = 0.95
num_id_sample_for_tuning = 200

In [19]:
sample_list = sample(list_pattern_py, num_id_sample_for_tuning)

In [20]:
dataframe_check = dataframe_weekly[sample_list].iloc[-2:].T

dataframe_check['Check'] = np.where((dataframe_check[271] == 0) &
                                    (dataframe_check[272] == 0), 1, 0)

sample_removed = dataframe_check[dataframe_check['Check'] == 1].index.tolist()

In [21]:
sample_list_filtered = [x for x in sample_list if (x not in sample_removed)]

In [22]:
len(sample_list_filtered)

158

# Hyperparameter tuning for Croston

In [23]:
train_data = dataframe_weekly[(dataframe_weekly['wm_yr_wk'] >= start_train_week) & (dataframe_weekly['wm_yr_wk'] <= end_train_week)]
test_data = dataframe_weekly.iloc[n_pred_weeks*-1:]

train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]

In [24]:
start = timer()

df_result_all_params = pd.DataFrame()
for product in sample_list_filtered:
    print('Currently Running: %s' % product)
    print('Progressing: {0} %'.format(round(sample_list_filtered.index(product) / len(sample_list_filtered) * 100, 2)))

    train_first = train_data_first_part[product]
    train_second = train_data_second_part[product]
    for i in list_params:
        predictions = list()
        history = [x for x in train_first[num_train_needed*-1:]]
        for t in train_second.index.tolist():
            yhat = Croston(history, extra_periods=1, alpha=i)['Forecast'].iloc[-1]
            predictions.append(yhat)
            history.append(train_second[t])
        df_result_temp = pd.DataFrame({'Product': [product for count in range(len(train_second))],
                                        'Actual Data': train_second,
                                        'Forecast': predictions,
                                        'Alpha': [i for count in range(len(train_second))]})
        df_result_all_params = df_result_all_params.append(df_result_temp, ignore_index=True)

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running: HOUSEHOLD_1_475_CA_4
Progressing: 0.0 %
Currently Running: HOUSEHOLD_2_036_CA_4
Progressing: 0.63 %
Currently Running: HOUSEHOLD_1_080_CA_3
Progressing: 1.27 %
Currently Running: HOUSEHOLD_1_241_CA_3
Progressing: 1.9 %
Currently Running: HOUSEHOLD_1_122_CA_2
Progressing: 2.53 %
Currently Running: HOUSEHOLD_1_471_CA_3
Progressing: 3.16 %
Currently Running: HOUSEHOLD_2_262_CA_3
Progressing: 3.8 %
Currently Running: HOUSEHOLD_1_047_CA_3
Progressing: 4.43 %
Currently Running: HOUSEHOLD_1_395_CA_4
Progressing: 5.06 %
Currently Running: HOUSEHOLD_1_387_CA_4
Progressing: 5.7 %
Currently Running: HOUSEHOLD_1_419_CA_1
Progressing: 6.33 %
Currently Running: HOUSEHOLD_2_007_CA_1
Progressing: 6.96 %
Currently Running: HOUSEHOLD_1_003_CA_3
Progressing: 7.59 %
Currently Running: HOUSEHOLD_1_517_CA_4
Progressing: 8.23 %
Currently Running: HOUSEHOLD_2_374_CA_1
Progressing: 8.86 %
Currently Running: HOUSEHOLD_1_018_CA_4
Progressing: 9.49 %
Currently Running: HOUSEHOLD_1_247_CA_1
Prog

In [25]:
df_result_all_params

,Product,Actual Data,Forecast,Alpha
0,HOUSEHOLD_1_475_CA_4,4.0,6.153875,0.05
1,HOUSEHOLD_1_475_CA_4,2.0,6.046983,0.05
2,HOUSEHOLD_1_475_CA_4,9.0,5.846064,0.05
3,HOUSEHOLD_1_475_CA_4,1.0,6.002701,0.05
4,HOUSEHOLD_1_475_CA_4,4.0,5.754163,0.05
...,...,...,...,...
22747,HOUSEHOLD_1_486_CA_1,3.0,6.700741,0.90
22748,HOUSEHOLD_1_486_CA_1,7.0,3.370074,0.90
22749,HOUSEHOLD_1_486_CA_1,6.0,6.637007,0.90
22750,HOUSEHOLD_1_486_CA_1,2.0,6.063701,0.90


# Summay all metrics

In [26]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [27]:
start = timer()

df_result_metrics_all_params = df_result_all_params.groupby(['Product', 'Alpha']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.0747619932500001 minutes


In [28]:
df_result_metrics_all_params

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_003_CA_3,0.05,1.539258,92.5869,170.7754,90.7837,8.795760,9.782903
1,HOUSEHOLD_1_003_CA_3,0.10,1.209253,72.7370,113.9069,66.1337,6.910017,7.845990
2,HOUSEHOLD_1_003_CA_3,0.15,0.756443,45.5003,62.1672,32.2944,4.322529,5.482974
3,HOUSEHOLD_1_003_CA_3,0.20,0.641077,38.5610,48.8976,24.9934,3.663297,4.690602
4,HOUSEHOLD_1_003_CA_3,0.25,0.638446,38.4028,46.3197,24.5557,3.648265,4.775831
...,...,...,...,...,...,...,...,...
2839,HOUSEHOLD_2_513_CA_3,0.70,1.238648,115.5258,134.7766,186.9084,12.563429,13.834249
2840,HOUSEHOLD_2_513_CA_3,0.75,1.237409,115.4103,136.3913,195.8114,12.550866,14.051786
2841,HOUSEHOLD_2_513_CA_3,0.80,1.233379,115.0344,138.0318,205.1298,12.509987,14.309780
2842,HOUSEHOLD_2_513_CA_3,0.85,1.226001,114.3462,139.7113,214.7513,12.435151,14.613911


# Check what products has unexpected metrics result

In [29]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [30]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [31]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [32]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [33]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [34]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [35]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [36]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 0.0 %
Percentage of unexpected values of WMAPE is: 0.0 %
Percentage of unexpected values of SMAPE is: 0.0 %
Percentage of unexpected values of MAPE is: 0.0 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [37]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

### Get MEAN metrics value of each Alpha

In [38]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.05,1.088795,56.549747,65.726611,61.040185,5.091418,5.887079
0.10,0.990452,53.116582,59.367289,58.551041,4.338717,5.179207
0.15,0.927423,51.268385,56.020096,57.384622,3.949145,4.781029
0.20,0.913162,50.977854,54.750370,57.413429,3.942479,4.773483
0.25,0.908170,51.051285,53.979316,57.546922,3.951035,4.773643
0.30,0.907560,51.287954,53.576532,57.868030,3.962329,4.784128
0.35,0.910081,51.621338,53.258758,58.199687,3.985851,4.806578
0.40,0.916271,52.091978,53.065566,58.551874,4.025108,4.839388
0.45,0.925309,52.714297,53.002180,58.931380,4.064380,4.881332


In [39]:
for mts in list_metrics:
    print('The optimum Alpha based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Alpha based on Mean MASE is: 0.3
The optimum Alpha based on Mean WMAPE is: 0.2
The optimum Alpha based on Mean SMAPE is: 0.45
The optimum Alpha based on Mean MAPE is: 0.15
The optimum Alpha based on Mean MAE is: 0.2
The optimum Alpha based on Mean RMSE is: 0.2


### Get MEDIAN metrics value of each Alpha

In [40]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.05,0.795435,42.79250,47.99215,47.82035,2.639156,3.240500
0.10,0.784651,41.42425,44.58390,45.58670,2.717804,3.208550
0.15,0.758143,40.22515,43.77690,43.25445,2.679717,3.090821
0.20,0.762781,39.67590,43.52465,42.88830,2.677317,3.098219
0.25,0.776092,39.58925,43.29175,42.28770,2.662885,3.117580
0.30,0.787496,39.81230,43.81265,43.10075,2.644637,3.165969
0.35,0.794337,40.23690,44.34765,43.67415,2.657040,3.172207
0.40,0.796109,41.36525,44.63625,44.78155,2.677915,3.175196
0.45,0.803152,42.29545,44.96030,45.01265,2.655014,3.209178


In [41]:
for mts in list_metrics:
    print('The optimum Alpha based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Alpha based on Median of MASE is: 0.15
The optimum Alpha based on Median of WMAPE is: 0.25
The optimum Alpha based on Median of SMAPE is: 0.25
The optimum Alpha based on Median of MAPE is: 0.25
The optimum Alpha based on Median of MAE is: 0.05
The optimum Alpha based on Median of RMSE is: 0.15


# Run Croston model after decide best paramaters

In [42]:
best_alpha = 0.35

In [43]:
dataframe_weekly

,wm_yr_wk,HOUSEHOLD_1_001_CA_3,HOUSEHOLD_1_002_CA_3,HOUSEHOLD_1_002_CA_4,HOUSEHOLD_1_003_CA_1,HOUSEHOLD_1_003_CA_2,HOUSEHOLD_1_003_CA_3,HOUSEHOLD_1_003_CA_4,HOUSEHOLD_1_004_CA_1,HOUSEHOLD_1_004_CA_3,...,HOUSEHOLD_2_510_CA_3,HOUSEHOLD_2_510_CA_4,HOUSEHOLD_2_511_CA_1,HOUSEHOLD_2_511_CA_4,HOUSEHOLD_2_512_CA_1,HOUSEHOLD_2_512_CA_2,HOUSEHOLD_2_512_CA_3,HOUSEHOLD_2_513_CA_3,HOUSEHOLD_2_513_CA_4,HOUSEHOLD_2_514_CA_1
0,11101,3.0,3.0,4.0,0.0,3.0,1.0,0.0,13.0,42.0,...,12.0,1.0,14.0,3.0,3.0,8.0,5.0,22.0,0.0,4.0
1,11102,2.0,6.0,3.0,0.0,6.0,6.0,7.0,3.0,17.0,...,13.0,2.0,5.0,2.0,4.0,6.0,15.0,38.0,2.0,0.0
2,11103,1.0,5.0,1.0,0.0,9.0,7.0,12.0,8.0,35.0,...,16.0,3.0,6.0,2.0,5.0,8.0,16.0,29.0,2.0,0.0
3,11104,4.0,7.0,0.0,0.0,2.0,8.0,4.0,11.0,23.0,...,0.0,0.0,5.0,3.0,4.0,19.0,5.0,14.0,2.0,1.0
4,11105,3.0,8.0,0.0,1.0,9.0,5.0,6.0,6.0,32.0,...,17.0,0.0,2.0,2.0,1.0,15.0,16.0,23.0,3.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,11608,4.0,9.0,2.0,8.0,5.0,7.0,3.0,8.0,25.0,...,19.0,2.0,4.0,7.0,3.0,4.0,3.0,23.0,0.0,0.0
269,11609,2.0,5.0,1.0,4.0,6.0,11.0,3.0,11.0,16.0,...,15.0,3.0,1.0,1.0,9.0,7.0,9.0,12.0,0.0,2.0
270,11610,12.0,4.0,3.0,4.0,10.0,8.0,9.0,11.0,21.0,...,17.0,4.0,10.0,6.0,1.0,11.0,8.0,17.0,4.0,1.0
271,11611,6.0,5.0,4.0,4.0,3.0,10.0,7.0,7.0,25.0,...,18.0,3.0,13.0,6.0,0.0,11.0,5.0,7.0,0.0,0.0


In [44]:
train_data = dataframe_weekly[(dataframe_weekly.wm_yr_wk >= start_train_week) & (dataframe_weekly.wm_yr_wk <= end_train_week)]
test_data = dataframe_weekly.iloc[n_pred_weeks*-1:]

In [45]:
df_result_best_params = pd.DataFrame()
for product in list_pattern_py:
    train_product = train_data[product]
    test_product = test_data[product]
    predictions = list()
    history = [x for x in train_product[num_train_needed*-1:]]
    for t in test_product.index.tolist():
        yhat = Croston(history, extra_periods=1, alpha=best_alpha)['Forecast'].iloc[-1]
        predictions.append(yhat)
        history.append(test_product[t])
    df_result_temp = pd.DataFrame({'Product': [product for count in range(len(test_product))],
                                    'Actual Data': test_product,
                                    'Forecast': predictions})
    df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True)

In [46]:
df_result_best_params

,Product,Actual Data,Forecast
0,HOUSEHOLD_1_003_CA_1,4.0,6.686017
1,HOUSEHOLD_1_003_CA_1,2.0,5.761773
2,HOUSEHOLD_1_004_CA_1,7.0,11.210684
3,HOUSEHOLD_1_004_CA_1,9.0,9.736945
4,HOUSEHOLD_1_005_CA_1,9.0,8.297443
...,...,...,...
4777,HOUSEHOLD_2_510_CA_4,1.0,3.069184
4778,HOUSEHOLD_2_511_CA_4,6.0,4.349474
4779,HOUSEHOLD_2_511_CA_4,3.0,4.926403
4780,HOUSEHOLD_2_513_CA_4,0.0,1.719550


In [47]:
df_result_final = df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [48]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_001_CA_3,0.690551,27.6221,26.0389,32.6028,1.381103,1.659006
1,HOUSEHOLD_1_002_CA_3,0.958284,82.1386,64.7991,118.7453,2.874852,3.118480
2,HOUSEHOLD_1_002_CA_4,0.588988,117.7975,137.0116,54.0270,2.355950,2.363996
3,HOUSEHOLD_1_003_CA_1,1.611947,107.4632,73.6012,127.6195,3.223895,3.268457
4,HOUSEHOLD_1_003_CA_2,inf,189.1334,96.1482,189.1334,5.674003,5.795209
...,...,...,...,...,...,...,...
2386,HOUSEHOLD_2_512_CA_2,1.196594,19.9432,21.8397,19.5901,2.393189,2.494722
2387,HOUSEHOLD_2_512_CA_3,0.531400,57.9709,63.0187,52.7093,6.376800,7.648715
2388,HOUSEHOLD_2_513_CA_3,0.522505,64.9600,72.2772,61.2399,12.017607,14.520369
2389,HOUSEHOLD_2_513_CA_4,inf,inf,200.0000,NaN,1.719550,1.719550


In [49]:
df_result_final.isnull().sum()

Product      0
MASE       414
WMAPE      414
SMAPE      418
MAPE       485
MAE          0
RMSE         0
dtype: int64

In [50]:
df_result_final.MAPE.mean()

58.991034155299054

In [51]:
df_result_final.to_csv('Croston_Smooth_Metrics.csv')